# Goals
The aim is to recreate FE versions of the test problems form the BICGStab paper and export them as mtx files.

## Set Up and Test
This is assembled from the Gridap tutorials.  It is set up to be as understandable as possible.  This just so that I can see where everything came from. 

The eigenfunctions and eigenvalues of $-\Delta u $ on the square domain $(0,\pi)^2$ with homogeneous Dirichlet conditions on all sides are 
$$
u_{i,j} = \sin(i x) s\sin(j y) \mbox{ with } \lambda_{i,j} = i^2 + j^2
$$
for integers $i$ and $j$.
This gives the sequence of smallest eigenvalues as 
$$
2,5,5,8,10,10,13,13,17,17,18,20,20,25, \dots 
$$



In [15]:
using Gridap, LinearAlgebra, LimitedLDLFactorizations, Preconditioners, SparseArrays

n = 30; Omega = (0.0,pi,0.0,pi);
Grid = CartesianDiscreteModel(Omega, (n,n))
# Grid has 4 corners and 4 edges
# "boundary" is all elements of the boundary of Omega
# Tags 1-4 are the corners and Tags 5-8 are the edges.
# Label each of the six sides differently
# boundary
Tags = get_face_labeling(Grid)
add_tag_from_tags!(Tags,"y=0",[1,2,5])
add_tag_from_tags!(Tags,"OtherEdges",[3,4,6,7,8])
# Piecewise linear Lagrange elements
order = 1; Elements = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(Grid,Elements;conformity=:H1,dirichlet_tags="boundary")
# Incorporating Homogeneous Dirichlet boundary conditions.
g(x) = 0.0
U = TrialFESpace(V,g)
# Triangulating and defining measures
degree = 2
Ω = Triangulation(Grid)
dΩ = Measure(Ω,degree)
# Setting up a simple test problem
f(x) = 1.0
a(u,v) = ∫( ∇(v)⋅∇(u) )*dΩ
b(v) = ∫( v*f )*dΩ
# Set up FE operator
op = AffineFEOperator(a,b,U,V)
# Extract Stiffness Matrix A
A=get_matrix(op)
# Compute Incomplete Cholesky Decomposition
# Dimensions needed to inhibit pivoting by specifying "natural order" 1:m 
# print(fieldnames(LimitedLDLFactorization))
m=size(A)[1]
LStructure = lldl(tril(A,-1), diag(A), 1:m, memory=2)
L=LStructure.L + I
D=spdiagm(LStructure.D)
# alpha should be zero and p should be the identity permutation
# \alpha then TAB produces pretty α
alpha=LStructure.α
p=LStructure.P
println("α=",alpha," p==1:m is ",p==1:m) 
# Check preconditioner quality
println("relative residual error L form is ", norm(A - L*D*L')/norm(A))
P=L*spdiagm(sqrt.(abs.(LStructure.D)));
println("relative residual error P form is ", norm(A - P*P')/norm(A))

α=0.0 p==1:m is true
relative residual error L form is 0.017302063030349046
relative residual error P form is 0.017302063030349046


In [18]:
λs=eigen(Matrix(A)).values; 
[λs[1:10] λs[m-9:m]]

10×2 Matrix{Float64}:
 0.0218724  3.94236
 0.0545014  3.94236
 0.0545014  3.9528
 0.0867729  3.9528
 0.108486   3.95677
 0.108486   3.95677
 0.140166   3.97464
 0.140166   3.97464
 0.183234   3.98543
 0.183234   3.98543

# Problem 1
$-\Delta u$
All Homogeneous Dirichlet conditions

# Problem 2
$-\nabla \cdot (D(x,y) *\nabla u)$
Homogeneous Dirichlet conditions only on y=0

# Problem 3
$-\nabla \cdot (D(x,y) *\nabla u)$
Homogeneous Dirichlet conditions only on y=0

# Problem 4
$-\nabla \cdot (D(x,y) *\nabla u)$
Homogeneous Dirichlet conditions only on y=0